In [ ]:
#Zielpfad für die csv Datei mit den Koordinaten -> /Users/furkan/Documents/Koordinaten_Algo.csv
#Zielpfad für Straßentyp -> /Users/furkan/Documents/strassenTyp_Algo.csv
#Formel für die Distanz-Berechnung (euklidische Distanz) ->  math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [19]:
import csv
import heapq
import math

class Stadt:
    def __init__(self, name, koordinaten=None):
        self.name = name
        self.koordinaten = koordinaten

class Strasse:
    def __init__(self, start, ziel, typ):
        self.start = start
        self.ziel = ziel
        self.typ = typ
        self.geschwindigkeit = 130 if typ == 'autobahn' else 80
        self.verbrauch = 9 if typ == 'autobahn' else 7

class Graph:
    def __init__(self):
        self.staedte = {}  # Stadtname: Stadtobjekt
        self.strassen = {} # Stadtname: Liste von Strassen

    def add_stadt(self, stadtname, koordinaten=None):
        if stadtname not in self.staedte:
            self.staedte[stadtname] = Stadt(stadtname, koordinaten)
    
    def add_strasse(self, start, ziel, typ):
        if start not in self.strassen:
            self.strassen[start] = []
        self.strassen[start].append(Strasse(start, ziel, typ))
        # Bidirektionale Verbindung
        if ziel not in self.strassen:
            self.strassen[ziel] = []
        self.strassen[ziel].append(Strasse(ziel, start, typ))

def euklidische_distanz(p1, p2):
    return math.sqrt((float(p1[0]) - float(p2[0]))**2 + (float(p1[1]) - float(p2[1]))**2)

def kuerzeste_route(distanz, strasse):
    return distanz

def schnellste_route(distanz, strasse):
    return distanz / strasse.geschwindigkeit

def energieeffizienteste_route(distanz, strasse):
    return distanz * strasse.verbrauch

def dijkstra(graph, start, ziel, kostenfunktion):
    queue = [(0, start, [])]
    besucht = set()
    while queue:
        (kosten, aktueller_knoten, pfad) = heapq.heappop(queue)
        if aktueller_knoten in besucht:
            continue
        if aktueller_knoten == ziel:
            return (kosten, pfad + [ziel])
        besucht.add(aktueller_knoten)
        for strasse in graph.strassen.get(aktueller_knoten, []):
            if strasse.ziel not in besucht:
                distanz = euklidische_distanz(graph.staedte[aktueller_knoten].koordinaten, graph.staedte[strasse.ziel].koordinaten)
                heapq.heappush(queue, (kosten + kostenfunktion(distanz, strasse), strasse.ziel, pfad + [aktueller_knoten]))
    return None

def lese_csv_und_erstelle_graph(koord_dateipfad, str_dateipfad, graph):
    try:
        with open(koord_dateipfad, mode='r', encoding='utf-8') as datei:
            reader = csv.reader(datei, delimiter=';')
            for zeile in reader:
                zeile = [feld for feld in zeile if feld]
                if zeile and len(zeile) == 3:
                    name, lat, lon = zeile
                    graph.add_stadt(name, (lat, lon))
    except FileNotFoundError:
        print(f"Die Datei {koord_dateipfad} wurde nicht gefunden.")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

    try:
        with open(str_dateipfad, mode='r', encoding='utf-8') as datei:
            reader = csv.reader(datei, delimiter=';')
            for zeile in reader:
                zeile = [feld for feld in zeile if feld]
                if zeile and len(zeile) == 3:
                    start, ziel, typ = zeile
                    graph.add_strasse(start, ziel, typ)
    except FileNotFoundError:
        print(f"Die Datei {str_dateipfad} wurde nicht gefunden.")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

graph = Graph()
pfad_koordinaten = '/Users/furkan/Documents/Koordinaten_Algo.csv'
pfad_strassen = '/Users/furkan/Documents/strassenTyp_Algo.csv'
lese_csv_und_erstelle_graph(pfad_koordinaten, pfad_strassen, graph)

beste_kuerzeste_route = None
beste_schnellste_route = None
beste_effizienteste_route = None

for start in graph.staedte:
    for ziel in graph.staedte:
        if start != ziel:
            kuerzeste = dijkstra(graph, start, ziel, kuerzeste_route)
            schnellste = dijkstra(graph, start, ziel, schnellste_route)
            effizienteste = dijkstra(graph, start, ziel, energieeffizienteste_route)

            if kuerzeste and (not beste_kuerzeste_route or kuerzeste[0] < beste_kuerzeste_route[0]):
                beste_kuerzeste_route = kuerzeste

            if schnellste and (not beste_schnellste_route or schnellste[0] < beste_schnellste_route[0]):
                beste_schnellste_route = schnellste

            if effizienteste and (not beste_effizienteste_route or effizienteste[0] < beste_effizienteste_route[0]):
                beste_effizienteste_route = effizienteste

print(f"Beste kürzeste Route: {beste_kuerzeste_route[0] / 10:.2f} Kilometer, zwischen dem Pfad: {beste_kuerzeste_route[1]}")
zeit_in_minuten = (beste_kuerzeste_route[0]/10)/130 * 60 #Distanz/10 wegen Kommastellen fehler / die Streckengeschwindigkeit um die Stunden zu haben * 60 für die Minuten
print(f"Beste schnellste Route: {zeit_in_minuten:.2f} Minuten, zwischen dem Pfad: {beste_schnellste_route[1]}")
effizienz_Liter = beste_kuerzeste_route[0]/10*9/100 #Ich nehme für die Rechnung die Distanz von der kürzesten Route, da die selbe Strecke überall die beste ist.
print(f"Beste energieeffizienteste Route: {effizienz_Liter:.2f} Liter, Pfad: {beste_effizienteste_route[1]}") 

Beste kürzeste Route: 6.82 Kilometer, zwischen dem Pfad: ['Bargteheide', 'Ahrensburg']
Beste schnellste Route: 3.15 Minuten, zwischen dem Pfad: ['Bargteheide', 'Ahrensburg']
Beste energieeffizienteste Route: 0.61 Liter, Pfad: ['Bargteheide', 'Ahrensburg']
